# Paired differences solution


In [ ]:
import numpy as np
import pandas as pd

rnd = np.random.default_rng()

df = pd.read_csv('data/hamilton.csv')
before = np.array(df['score_before'])
after = np.array(df['score_after'])

observed_diff = np.mean(after) - np.mean(before)

# Let us start with a permutation test.
both = np.concatenate([before, after])
n_before = len(before)

# Samples in the null world.
n_trials = 10_000
results = np.zeros(n_trials)
for i in range(n_trials):
    shuffled = rnd.permuted(both)
    fake_before = shuffled[:n_before]
    fake_after = shuffled[n_before:]
    fake_diff = np.mean(fake_after) - np.mean(fake_before)
    results[i] = fake_diff

# We are interested in fake differences that are larger
# in magnitude than the observed difference (hence "abs").
# Here we have no prior hypothesis about which direction the difference
# will go.
k = np.sum(np.abs(results) >= np.abs(observed_diff))
kk = k / n_trials
print('Permutation p null-world abs >= abs observed:', kk)

In [ ]:
# Next a bootstrap test.
n_after = len(after)  # Of course, in our case, this will be == n_before
results = np.zeros(n_trials)
for i in range(n_trials):
    fake_before = rnd.choice(both, size=n_before)
    fake_after = rnd.choice(both, size=n_after)
    fake_diff = np.mean(fake_after) - np.mean(fake_before)
    results[i] = fake_diff

k = np.sum(np.abs(results) >= np.abs(observed_diff))
kk = k / n_trials
print('Bootstrap p null-world abs >= abs observed:', kk)

Finally we consider the pairs. Here we *do* take the pairs into account.
We have some reason to think that the patients or cars vary in some
substantial way in their level of depression, or their tendency to break
down, but we believe that the patients’ *response* to treatment or the
difference between the mechanics is the value of interest.

In that case, we are interested in the *differences* between the pairs.
In the null world, these before / after (mechanic A / mechanic B)
differences are random. In the null-world, where there is no difference
between before/after or mechanics 1 and 2, we can flip the before /
after (A / B) pairs and be in the same world.

Notice that flipping the before / after or A / B in the pair just
changes the sign of the difference.

So we will simulate the effect of flipping the values in the pair, by
choosing a random sign for the pair, where -1 means pair is flipped, and
1 means pair is in original order. We recalculated the mean difference
with these random signs (flips) applied, and these will be our values in
the null-world.

In [ ]:
# A test of paired difference with sign flips for the null world.
differences = after - before
observed_mdiff = np.mean(differences)
n_both = len(differences)

results = np.zeros(n_trials)
for i in range(n_trials):
    # Choose random signs to perform random flips of the pairs.
    signs = rnd.choice([-1, 1], size=n_both)
    # Do flips.
    fake_differences = signs * differences
    # Calculate mean difference and store result.
    results[i] = np.mean(fake_differences)

k = np.sum(np.abs(results) >= np.abs(observed_mdiff))
kk = k / n_trials
print('Sign-flip p null-world abs >= abs observed:', kk)

Notice that the sign-flip test, in which we preserve the information
about the patients / cars, is much more convincing than the permutation
or bootstrap tests above, where we choose to ignore that information.

This can occur when the values within the pairs (rows) are similar to
each other, but less similar across different pairs (rows).